In [3]:
import pandas as pd
import datetime as dt
import smtplib
import email.message
import pdf_mail

In [4]:
#!pip install pdf-mail

# Lendo arquivo excel

In [5]:
tabela = pd.read_excel('devedores.xlsx')

In [6]:
display(tabela)

,CPF,Valor_em_aberto,Data_Prevista,Status,Email,NF
0,10978459522,500,2023-11-27,Em aberto,fake_email1@gmail.com,197
1,12564375375,275,2023-12-31,Pago,fake_email2@gmail.com,128
2,24864692469,8000,2023-10-28,Em aberto,fake_email3@gmail.com,198
3,19351750353,524,2023-07-12,Em aberto,fake_email4@gmail.com,129
4,46247357486,9240,2023-10-27,Em aberto,fake_email5@gmail.com,131
5,58135180381,7777,2022-06-05,Em aberto,fake_email6@gmail.com,199


In [7]:
tabela.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CPF              6 non-null      int64         
 1   Valor_em_aberto  6 non-null      int64         
 2   Data_Prevista    6 non-null      datetime64[ns]
 3   Status           6 non-null      object        
 4   Email            6 non-null      object        
 5   NF               6 non-null      int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 416.0+ bytes


# Verificando data de hoje

In [8]:
hoje = dt.datetime.now()

In [9]:
print(hoje)

2023-10-29 15:02:13.216954


# Coletando apenas os dados de clientes devendo

In [10]:
tabela_devedores = tabela.loc[tabela['Status'] == 'Em aberto']

In [11]:
display(tabela_devedores)

,CPF,Valor_em_aberto,Data_Prevista,Status,Email,NF
0,10978459522,500,2023-11-27,Em aberto,fake_email1@gmail.com,197
2,24864692469,8000,2023-10-28,Em aberto,fake_email3@gmail.com,198
3,19351750353,524,2023-07-12,Em aberto,fake_email4@gmail.com,129
4,46247357486,9240,2023-10-27,Em aberto,fake_email5@gmail.com,131
5,58135180381,7777,2022-06-05,Em aberto,fake_email6@gmail.com,199


In [12]:
pd.to_datetime(tabela_devedores['Data_Prevista'])
tabela_devedores['Valor_em_aberto'] = tabela_devedores['Valor_em_aberto'].astype('float64')

In [13]:
tabela_devedores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 5
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CPF              5 non-null      int64         
 1   Valor_em_aberto  5 non-null      float64       
 2   Data_Prevista    5 non-null      datetime64[ns]
 3   Status           5 non-null      object        
 4   Email            5 non-null      object        
 5   NF               5 non-null      int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 280.0+ bytes


In [14]:
tabela_devedores = tabela_devedores.loc[tabela_devedores['Data_Prevista'] < hoje]

In [15]:
tabela_devedores

,CPF,Valor_em_aberto,Data_Prevista,Status,Email,NF
2,24864692469,8000.0,2023-10-28,Em aberto,fake_email3@gmail.com,198
3,19351750353,524.0,2023-07-12,Em aberto,fake_email4@gmail.com,129
4,46247357486,9240.0,2023-10-27,Em aberto,fake_email5@gmail.com,131
5,58135180381,7777.0,2022-06-05,Em aberto,fake_email6@gmail.com,199


# Enviar e-mail

In [16]:
dados = tabela_devedores[['Valor_em_aberto', 'Data_Prevista', 'Email', 'NF']].values.tolist()

In [ ]:
password = 'GMAIL PASSWORD HERE'
remetente = 'EMAIL_HERE'
for dado in dados:
    valor = dado[0]
    prazo = dado[1]
    prazo = prazo.strftime('%d/%m/%Y')
    destinatario = dado[2]
    nf = dado[3]

    corpo_email = f"""
    Verificamos um atraso no pagamento referente a NF {nf}, previsto para {prazo}, no valor de R${valor:.2f}
    """

    k = pdf_mail.sendpdf(remetente, destinatario, password, 'Débito em atraso!', corpo_email, 'nota_fiscal.pdf', 'C:/Users/eului/OneDrive/Área de Trabalho/cobranca')
    k.email_send()
    
    print('Cobrança enviada com sucesso!')